## Segments

* NAMES - explains the formats
* CMPLDR.DAT - company leader
* CMPOPN.DAT - campaign oponent
* COMPANY.DAT - ????
* DTCH - detachment 
* DTCHLDR.DAT - detachment leader
* SUPPORT.DAT - detachment makeups
* UNIT.DAT - individual unit
* WEAPON.DAT - weapon stats

regiment has 4 companies
company has leader and 4 detachments
detachment has leader name 1-5 units
unit has 1-4 weapons?

Action Sets? `./SET/*.BIN` and `SET.MUK`

* REGION 1-36

## action sets

describe which set of sprites refer to a certain action, game files has a couple of action set BIN files, the rest are merged into one big SET.MUK file

## buildings

the building files are merged into one big BDG.MUK file



## regions

`mapnum.pcx` with region map and order:

{
    region name,
    resources,
    region info text filename,
    region map name file,
    x,
    y,
    icon coordinates,
    [0-7] neighbouring regions (-1 for empty),
    [0-23] x, y, facing, form
)

where is the pcx pallet index value kept?

## Formats

Company leader {
    name, 
    experience,
    detachement1,
    detachement2,
    detachement3,
    detachement4
}

detachment 1-4 has nationality and unit type.

detachment leader {
    nationality,
    name
}

detachment
{
	ID.
	"Title Line One".
   "Title Line Two".
	NUM_UNIT1 UNIT_ID1.
	...
	NUM_UNIT5 UNIT_ID5.
	Break. Morale. VictoryPoints.
	Cost.
	LIconOffset. RIconOffset.NationalityOffset.
	Flags.
   AllowableNationalityFlag.
   AllowTypeFlag.
   StartExperience.
   Turns(time to build). !!!!!for now is   Cost / 200
}

In [33]:
import os
from collections import defaultdict
import logging
from IPython.display import HTML, display
import math
from itertools import islice, product

In [34]:
nationalities = {
    # in the hpp these had an extra WORD with zeros?
    # used in the company ledaers but not the detach leaders
    0x0001: "FLAG_NATION_TALLARN",
    0x0002: "FLAG_NATION_MORDIAN",
    0x0004: "FLAG_NATION_CATACHAN",
    0x0007: "FLAG_NATION_T_M_C",
    0x0008: "FLAG_NATION_ATTILAN",
    0x0010: "FLAG_NATION_SPACEMARINE",
    0x0020: "FLAG_NATION_FORGEWORLD",
    0x0040: "FLAG_NATION_PLANETARY",
    0x00FF: "FLAG_NATION_ANYIMPERIAL",
    0x0100: "FLAG_NATION_GOFF",
    0x0200: "FLAG_NATION_BAD_MOON",
    0x0400: "FLAG_NATION_EVIL_SUNZ",
    0x0800: "FLAG_NATION_MEKBOY",
    0x0C00: "FLAG_NATION_ES_OR_MEK",
    0x0900: "FLAG_NATION_GOFF_OR_MEK",
    0x0a00: "FLAG_NATION_BAD_OR_MEK",
    0xff00: "FLAG_NATION_ANYORK",
    0xffff: "FLAG_NATION_ANYNATION",
}

unit_types = {
    0xffffffff: "FLAG_TYPE_ANYTYPE",
    0x00010000: "FLAG_TYPE_INFANTRY_WALKER",
    0x00020000: "FLAG_TYPE_CAVALRY",
    0x00100000: "FLAG_TYPE_KNIGHTS",
    0x00080000: "FLAG_TYPE_TANKS",
    0x00040000: "FLAG_TYPE_SUPER_HEAVY",
    0x000c0000: "FLAG_TYPE_ANY_ARMOR",
    0x00200000: "FLAG_TYPE_TITAN",
    0x00400000: "FLAG_TYPE_FLYER",
    0x00800000: "FLAG_TYPE_BIKES",
    0x00880000: "FLAG_TYPE_TANK_OR_BIKE",
    0x008c0000: "FLAG_TYPE_ANY_TANK_OR_BIKE"
}

In [38]:
def chunk(it, size):
    it = iter(it)
    return list(iter(lambda: tuple(islice(it, size)), ()))


def format_word(v, w):
    text = '{:02X} {:02X}'.format(v, w)
    return f'{text}'

    
def hex_view(data):
    text = []
    text.append(
        14 * '&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8)]) +
        '&nbsp;&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8, 16)])
    
    )
    text.append('')
    for i, x in enumerate(chunk(data, 16)):
        text.append('{:08X}:&nbsp;&nbsp;&nbsp;&nbsp;'.format(i * 16) +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[:8:2], x[1:8:2])]) +
              '&nbsp;&nbsp;' +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[8:16:2], x[9:16:2])])
             )
    return '<span style="font-family: Courier New, Courier, Lucida Sans Typewriter, Lucida Typewriter, monospace;">{}</span>'.format("<br>".join(text))


def display_hex_view(data):
    display(HTML(hex_view(data)))


display_hex_view(bytes(range(132)))

In [54]:
with open(f'{FOLDER}/DTCHLDR.DAT', 'rb') as file:
    leader_count = int.from_bytes(file.read(2), 'little')
    blank = file.read(4)
    deatchment_leaders = []
    for i in range(leader_count):
        nation = int.from_bytes(file.read(2), 'little')
        deatchment_leaders.append((
            nationalities.get(nation, hex(nation)), 
            bytes([x for x in file.read(26) if x != 0x00]).decode('utf-8')
        ))
for x in deatchment_leaders:
    print(x)


('FLAG_NATION_TALLARN', 'Arif')
('FLAG_NATION_TALLARN', 'Abdul Salam')
('FLAG_NATION_TALLARN', 'Azhara')
('FLAG_NATION_TALLARN', 'Rahimat')
('FLAG_NATION_TALLARN', 'Asim')
('FLAG_NATION_TALLARN', 'Tahir Majnun')
('FLAG_NATION_TALLARN', 'Jabir')
('FLAG_NATION_MORDIAN', 'Fredrick')
('FLAG_NATION_MORDIAN', 'Ulrich')
('FLAG_NATION_MORDIAN', 'Dietrich')
('FLAG_NATION_MORDIAN', 'Hans Bernd')
('FLAG_NATION_MORDIAN', 'Claus Schenk')
('FLAG_NATION_MORDIAN', 'Kluge')
('FLAG_NATION_MORDIAN', 'Konstantin')
('FLAG_NATION_CATACHAN', 'Pelayo')
('FLAG_NATION_CATACHAN', 'Rodrigo')
('FLAG_NATION_CATACHAN', 'Diaz')
('FLAG_NATION_CATACHAN', 'Jimena')
('FLAG_NATION_CATACHAN', 'Comuneros')
('FLAG_NATION_CATACHAN', 'Acuna')
('FLAG_NATION_CATACHAN', 'Giron')
('FLAG_NATION_SPACEMARINE', 'Theonoe')
('FLAG_NATION_SPACEMARINE', 'Creon')
('FLAG_NATION_SPACEMARINE', 'Andros')
('FLAG_NATION_SPACEMARINE', 'Sybaris')
('FLAG_NATION_SPACEMARINE', 'Dorian')
('FLAG_NATION_SPACEMARINE', 'Iphicrates')
('FLAG_NATION_SPACEMA

In [55]:
with open(f'{FOLDER}/CMPLDR.DAT', 'rb') as file:
    leader_count = int.from_bytes(file.read(4), 'little')
    print(leader_count, leader_count * (24 + 49))
    for i in range(leader_count):
        name = bytes([x for x in file.read(24) if x != 0x00]).decode('utf-8')
        exp = int.from_bytes(file.read(1), 'little')
        data = []
        for i in range(4):
            n = int.from_bytes(file.read(4), 'little')
            data.append({
                'nation': nationalities.get(n >> 16, hex(n >> 16))
            })
        for i in range(4):
            u = int.from_bytes(file.read(4), 'little')
            data[i]['unit_type'] = unit_types.get(u, hex(u))
        for i in range(4):
            default = int.from_bytes(file.read(4), 'little')
            data[i]['default_unit'] = default
        print(name, exp)
        for x in data:
            print(x)
        print()

60 4380
Horgrawl 10
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 11152}
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 11130}
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 11130}
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 11128}

Jericho 6
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_INFANTRY_WALKER', 'default_unit': 11163}
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_ANY_ARMOR', 'default_unit': 11122}
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 11108}
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 11128}

Pyrhus 5
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_INFANTRY_WALKER', 'default_unit': 11163}
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_TANKS', 'default_unit': 11108}
{'nation': 'FLAG_

In [58]:
# Look at the top of the files
FOLDER = '/home/alex/.wine/drive_c/GOG Games/WH40K Final Liberation/DAT'
OUTPUT = './sprites/'

for filepath in sorted(os.listdir(FOLDER)):
    filesize = os.path.getsize(FOLDER + '/' + filepath)
    filename = filepath.split('.')[0]
    print(filename, filesize)
    if filename == 'UNIT':
        with open(f'{FOLDER}/{filepath}', 'rb') as file:
            header = file.read(13)
            count = int.from_bytes(file.read(4), 'little')
            for i in range(count):
                uid = int.from_bytes(file.read(4), 'little')
                data2 = int.from_bytes(file.read(4), 'little')
                if i == 0:
                    data3 = int.from_bytes(file.read(4), 'little')
                data4 = file.read(1)
                name_length = int.from_bytes(file.read(1), 'little')
                name = bytes(file.read(name_length)).decode('utf-8')
                move = int.from_bytes(file.read(1), 'little')
                save = int.from_bytes(file.read(1), 'little')
                caf = int.from_bytes(file.read(1), 'little')
                armours = []
                for i in range(4):
                    unknown = int.from_bytes(file.read(1), 'little')
                    assert unknown == 0, "Initial armour value should be zero"
                    d6_value_legs = int.from_bytes(file.read(1), 'little')
                    armour_front = int.from_bytes(file.read(1), 'little')
                    armour_side = int.from_bytes(file.read(1), 'little')
                    armour_rear = int.from_bytes(file.read(1), 'little')
                    hit_offset_x = int.from_bytes(file.read(2), 'little', signed=True)
                    hit_offset_y = int.from_bytes(file.read(2), 'little', signed=True)
                    hit_offset_z = int.from_bytes(file.read(2), 'little', signed=True)
                    armours.append(
                        (unknown, d6_value_legs, armour_front, armour_side, armour_rear, 
                         hit_offset_x, hit_offset_y, hit_offset_z))
                chassis_type = int.from_bytes(file.read(1), 'little')
                weapon_slots = int.from_bytes(file.read(1), 'little')
                shield_type = int.from_bytes(file.read(1), 'little')
                shields = int.from_bytes(file.read(1), 'little')
                hit_points = int.from_bytes(file.read(1), 'little')
                unit_cost = int.from_bytes(file.read(2), 'little')
                unknown = int.from_bytes(file.read(8), 'little')
                # if the crit damage flag is set
                crit_type = int.from_bytes(file.read(1), 'little')
                dx = int.from_bytes(file.read(1), 'little', signed=True)
                dy = int.from_bytes(file.read(1), 'little', signed=True)
                dz = int.from_bytes(file.read(1), 'little', signed=True)
                crits = []
                weapons = []
                for i in range(11):
                    crits.append(file.read(4))
                for i in range(weapon_slots):
                    unknown = file.read(2)
                for i in range(weapon_slots):
                    weapons.append(
                        {'weapon_count': int.from_bytes(file.read(1), 'little')}
                    )
                for i in range(weapon_slots):
                    weapons[i]['base_hit_chance'] = int.from_bytes(file.read(1), 'little')
                for i in range(weapon_slots):
                    weapons[i]['base_hit_chance2'] = int.from_bytes(file.read(1), 'little')
                for i in range(weapon_slots):
                    weapons[i]['dx'] = int.from_bytes(file.read(1), 'little', signed=True)
                    weapons[i]['dy'] = int.from_bytes(file.read(1), 'little', signed=True)
                    weapons[i]['dz'] = int.from_bytes(file.read(1), 'little', signed=True)
                    term = int.from_bytes(file.read(1), 'little', signed=True)
                    assert term == 0, "final byte in weapon slot should be zero"
                unknown = file.read(6)
                #display_hex_view(file.read(128))
                print("id", uid)
                print(hex(data2), hex(data3), data4, name)
                print("move", move, "save", save, "caf", caf)
                print("armour", armours)
                print("chassis type", chassis_type)
                print("shield", shields)
                print("hits", hit_points)
                print("unit cost", unit_cost)
                print("crit type", crit_type)
                print(crits)
                print("weapons", weapons)

CMPLDR 4384
CMPOPN 404479
COMPANY 1387
DTCHLDR 2300
SUPPORT 5102
UNIT 12073
id 1504
0x40b70 0x1f4000 b'\x00' Warlord
move 6 save 0 caf 14
armour [(0, 2, 90, 75, 60, -30, -10, 20), (0, 3, 120, 90, 60, 0, -30, 80), (0, 4, 100, 80, 60, -35, -30, 90), (0, 6, 85, 75, 65, -20, -20, 100)]
chassis type 15
shield 6
hits 12
unit cost 600
crit type 255
[b'\x02\x01\x02\x05', b'\x02\x01\x02\x05', b'\x02\x01\x00\x03', b'\x03\x00\x00\x03', b'\x02\x00\x01\x03', b'\x01\x00\x01\x03', b'\x02\x00\x02\x03', b'\x02\x01\x01\x02', b'\x02\x01\x03\x04', b'\x03\x00\x03\x04', b'\x03\x01\x03\x04']
weapons [{'weapon_count': 1, 'base_hit_chance': 50, 'base_hit_chance2': 216, 'dx': 0, 'dy': 0, 'dz': 0}, {'weapon_count': 1, 'base_hit_chance': 50, 'base_hit_chance2': 211, 'dx': -33, 'dy': -35, 'dz': 125}, {'weapon_count': 1, 'base_hit_chance': 50, 'base_hit_chance2': 90, 'dx': 28, 'dy': -35, 'dz': 125}, {'weapon_count': 1, 'base_hit_chance': 50, 'base_hit_chance2': 0, 'dx': 2, 'dy': 0, 'dz': 3}]


AssertionError: final byte in weapon slot should be zero